<a href="https://colab.research.google.com/github/YasirHabib/Deep-Learning-Convolutional-Neural-Networks-in-Python/blob/master/svhn_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf

In [0]:
def init_filter(shape):
  w = np.random.randn(*shape) * np.sqrt(2.0 / np.prod(shape[1:]))
  return w.astype(np.float32)

In [0]:
def convpool(X, W, b):
  conv_out = tf.nn.conv2d(X, W, strides=[1, 1, 1, 1], padding='SAME')
  conv_out = tf.nn.bias_add(conv_out, b)
  pool_out = tf.nn.max_pool(conv_out, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
  return tf.nn.relu(pool_out)

In [0]:
class HiddenLayer():
  def __init__(self, D, M):
    W = np.random.randn(D, M) / np.sqrt(D)
    b = np.zeros(M)
    
    self.W = tf.Variable(W.astype(np.float32))
    self.b = tf.Variable(b.astype(np.float32))
    
    self.params = [self.W, self.b]
    
  def HiddenLayer_forward(self, X):
    return tf.nn.relu(tf.matmul(X, self.W)+self.b)

In [0]:
class FinalLayer():
  def __init__(self, M, K):
    W = np.random.randn(M, K) / np.sqrt(M)
    b = np.zeros(K)
    
    self.W = tf.Variable(W.astype(np.float32))
    self.b = tf.Variable(b.astype(np.float32))
    
    self.params = [self.W, self.b]
    
  def FinalLayer_forward(self, Z):
    return tf.matmul(Z, self.W)+self.b

In [0]:
class ANN():
  def __init__(self, size):
    self.size = size
    
  def fit(self, Xtrain, Ytrain, Xtest, Ytest, lr=1e-1, reg=0.01, training_epochs=10, batch_sz=500):
    N = Xtrain.shape[0]
    K = len(set(Ytrain))
    pool_sz = (2, 2)
    
    W1_shape = (5, 5, 3, 20) # (filter_width, filter_height, num_color_channels, num_feature_maps)
    W1 = init_filter(W1_shape)
    b1 = np.zeros(W1_shape[-1], dtype = np.float32)
    
    W2_shape = (5, 5, 20, 50)
    W2 = init_filter(W2_shape)
    b2 = np.zeros(W2_shape[-1], dtype = np.float32)
    
    X = tf.placeholder(tf.float32)
    Y = tf.placeholder(tf.int32)
    W1 = tf.Variable(W1)
    b1 = tf.Variable(b1)
    W2 = tf.Variable(W2)
    b2 = tf.Variable(b2)
    
    self.parameters = [W1,b1,W2,b2]
    
    self.layers = []
    h = HiddenLayer(W2_shape[-1]*8*8, self.size)
    self.layers.append(h)
    
    f = FinalLayer(self.size, K)
    self.layers.append(f)
    
    for obj in self.layers:
      self.parameters += obj.params
      
    # forward pass
    Z1 = convpool(X, W1, b1)
    Z2 = convpool(Z1, W2, b2)
    

In [0]:
def main():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount = True)
  
  train = scipy.io.loadmat('/content/gdrive/My Drive/Colab Notebooks/Convolutional Neural Networks/train_32x32.mat')
  test = scipy.io.loadmat('/content/gdrive/My Drive/Colab Notebooks/Convolutional Neural Networks/test_32x32.mat')
  
  Xtrain = train['X']
  Ytrain = train['y']
  #print(Xtrain.shape)           (32, 32, 3, 73257)
  #print(Ytrain.shape)           (73257, 1)
  Ytrain = Ytrain.flatten() - 1
  
  Xtest = test['X']
  Ytest = test['y']
  #print(Xtest.shape)            (32, 32, 3, 26032)
  #print(Ytest.shape)            (26032, 1)
  Ytest = Ytest.flatten() - 1
  
  Xtrain = (Xtrain.transpose(3,0,1,2) / 255).astype(np.float32)
  Xtest = (Xtest.transpose(3,0,1,2) / 255).astype(np.float32)
  
  model = ANN(500)
  model.fit(Xtrain, Ytrain, Xtest, Ytest)
  
if __name__ == '__main__':
  main()